In [0]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/FileStore/tables/online_retail_dataset.csv")\
  .coalesce(5)
df.cache()
df.createOrReplaceTempView("dfTable")


In [0]:
display(df.head(5))

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [0]:
from pyspark.sql.functions import count
df.select(count("StockCode")).show() # 541909


+----------------+
count(StockCode)|
+----------------+
 541909|
+----------------+

In [0]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct("StockCode")).show() # 4070


+-------------------------+
count(DISTINCT StockCode)|
+-------------------------+
 4070|
+-------------------------+

For a large data set, approximate distinct count is helpful:

In [0]:
from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("StockCode", 0.1)).show() # 3364


+--------------------------------+
approx_count_distinct(StockCode)|
+--------------------------------+
 3364|
+--------------------------------+

In [0]:
from pyspark.sql.functions import first, last
df.select(first("StockCode"), last("StockCode")).show()


+----------------+---------------+
first(StockCode)|last(StockCode)|
+----------------+---------------+
 85123A| 22138|
+----------------+---------------+

In [0]:
from pyspark.sql.functions import min, max
df.select(min("Quantity"), max("Quantity")).show()


+-------------+-------------+
min(Quantity)|max(Quantity)|
+-------------+-------------+
 -80995| 80995|
+-------------+-------------+

In [0]:
from pyspark.sql.functions import sum
df.select(sum("Quantity")).show() # 5176450


+-------------+
sum(Quantity)|
+-------------+
 5176450|
+-------------+

Summing just the distinct values:

In [0]:
from pyspark.sql.functions import sumDistinct
df.select(sumDistinct("Quantity")).show() # 29310


+----------------------+
sum(DISTINCT Quantity)|
+----------------------+
 29310|
+----------------------+

Aliasing a average function to reuse columns later:

In [0]:
from pyspark.sql.functions import sum, count, avg, expr

df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases"))\
  .selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases").show()


+--------------------------------------+----------------+----------------+
(total_purchases / total_transactions)| avg_purchases| mean_purchases|
+--------------------------------------+----------------+----------------+
 9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+

Spark has both the formula for the sample standard deviation as well as the formula for the population standard deviation. These are fundamentally different statistical formulae By default, Spark performs the formula forthe sample standard deviation or variance if you use the variance or stddev functions

Explicitly refer to the population  or standard deviation or variance:

In [0]:
from pyspark.sql.functions import var_pop, stddev_pop
from pyspark.sql.functions import var_samp, stddev_samp
df.select(var_pop("Quantity"), var_samp("Quantity"),
  stddev_pop("Quantity"), stddev_samp("Quantity")).show()


+------------------+------------------+--------------------+---------------------+
 var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
47559.303646609056|47559.391409298754| 218.08095663447796| 218.08115785023418|
+------------------+------------------+--------------------+---------------------+

Skewness - measures the asymmetry of the values in your data around the mean

Kurtosis - measures the tail of data 


Both measure the extremities of your data. Both relevant specifically when modeling your data as a probability distribution of a random variable.

In [0]:
from pyspark.sql.functions import skewness, kurtosis
df.select(skewness("Quantity"), kurtosis("Quantity")).show()


+-------------------+------------------+
 skewness(Quantity)|kurtosis(Quantity)|
+-------------------+------------------+
-0.2640755761052562|119768.05495536952|
+-------------------+------------------+

**covariance and correlation**

Correlation measures the Pearson correlation coefficient, which is scaled between –1 and +1. 

Covariance is scaled according to the inputs in the data - it is measure of the joint variability of two random variables. Like the var function, covariance can be calculated either as the sample covariance or the population covariance.  Correlation has no notion of this and therefore does not have calculations for population orsample

In [0]:
from pyspark.sql.functions import corr, covar_pop, covar_samp
df.select(corr("InvoiceNo", "Quantity"), covar_samp("InvoiceNo", "Quantity"),
    covar_pop("InvoiceNo", "Quantity")).show()


+-------------------------+-------------------------------+------------------------------+
corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
 4.912186085635685E-4| 1052.7280543902734| 1052.7260778741693|
+-------------------------+-------------------------------+------------------------------+

Spark can perform aggregations, not just of numerical values using formulas, but on complex types. Can collect a list of values present in a given column, or only the unique values, by collecting to a set. You can use this to define programmatic access later on in the pipeline or pass the entire collection in a user-defined function (UDF)

In [0]:
from pyspark.sql.functions import collect_set, collect_list
df.agg(collect_set("Country"), collect_list("Country")).show()


+--------------------+---------------------+
collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+

Group by each unique invoice number and get the count of items on that invoice - this returns another DataFrame and is lazily performed. 

We do this grouping in two phases
* First we specify the column(s) on which we would like to group -  returns a RelationalGroupedDataset
* Then we specify the aggregation(s) - returns a DataFrame.

In [0]:
df.groupBy("InvoiceNo","CustomerId").count().show()

+---------+----------+-----+
InvoiceNo|CustomerId|count|
+---------+----------+-----+
 536846| 14573| 76|
 537026| 12395| 12|
 537883| 14437| 5|
 538068| 17978| 12|
 538279| 14952| 7|
 538800| 16458| 10|
 538942| 17346| 12|
 C539947| 13854| 1|
 540096| 13253| 16|
 540530| 14755| 27|
 541225| 14099| 19|
 541978| 13551| 4|
 542093| 17677| 16|
 536596| null| 6|
 537252| null| 1|
 538041| null| 1|
 537159| 14527| 28|
 537213| 12748| 6|
 538191| 15061| 16|
 C539301| 13496| 1|
+---------+----------+-----+
only showing top 20 rows

Rather than passing that function as an expression into a select statement, we specify it as within agg. This makes it possible to pass in arbitrary expressions that have an aggregation specified. You can even do things like alias a column after transforming it for later use in your data flow:

In [0]:
from pyspark.sql.functions import count

test = df.groupBy("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)")).show()

display(test)


+---------+----+---------------+
InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
 536596| 6| 6|
 536938| 14| 14|
 537252| 1| 1|
 537691| 20| 20|
 538041| 1| 1|
 538184| 26| 26|
 538517| 53| 53|
 538879| 19| 19|
 539275| 6| 6|
 539630| 12| 12|
 540499| 24| 24|
 540540| 22| 22|
 C540850| 1| 1|
 540976| 48| 48|
 541432| 4| 4|
 541518| 101| 101|
 541783| 35| 35|
 542026| 9| 9|
 542375| 6| 6|
 536597| 28| 28|
+---------+----+---------------+
only showing top 20 rows

Can specify your transformations as a series of Maps for which the key is the column, and the value is the aggregation function (as a string) that you would like to perform. You can reuse multiple column names if you specify them inline.

In [0]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)"))\
  .show()


+---------+------------------+--------------------+
InvoiceNo| avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
 536596| 1.5| 1.1180339887498947|
 536938|33.142857142857146| 20.698023172885524|
 537252| 31.0| 0.0|
 537691| 8.15| 5.597097462078001|
 538041| 30.0| 0.0|
 538184|12.076923076923077| 8.142590198943392|
 538517|3.0377358490566038| 2.3946659604837897|
 538879|21.157894736842106| 11.811070444356483|
 539275| 26.0| 12.806248474865697|
 539630|20.333333333333332| 10.225241100118645|
 540499| 3.75| 2.6653642652865788|
 540540|2.1363636363636362| 1.0572457590557278|
 C540850| -1.0| 0.0|
 540976|10.520833333333334| 6.496760677872902|
 541432| 12.25| 10.825317547305483|
 541518| 23.10891089108911| 20.550782784878713|
 541783|11.314285714285715| 8.467657556242811|
 542026| 7.666666666666667| 4.853406592853679|
 542375| 8.0| 3.4641016151377544|
 536597|2.5357142857142856| 2.7448932175059566|
+---------+------------------+--------------------+
only showing top 20 rows

Add a date column that will convert our invoice date into a column that contains only date information (not time information, too):

In [0]:
from pyspark.sql.functions import col, to_date
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")


The first step to a window function is to create a window specification. Note that the partitionby is unrelated to the partitioning scheme concept that we have covered thus far. The ordering determines the ordering within a given partition, and, finally, the frame specification (the rowsBetween statement) states which rows will be included in the frame based on its reference to the currentinput row. In the following example, we look at all previous rows up to the current row:

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc
windowSpec = Window\
  .partitionBy("CustomerId", "date")\
  .orderBy(desc("Quantity"))\
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)




Use an aggregation function to learn more about each specific customer, establishing the maximum purchase quantity over all time. Use the same aggregation functions that we saw earlier, by passing a column name or expression, indicating the window specification that defines to which frames of data this function will apply:

In [0]:
from pyspark.sql.functions import max
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

# display(print(maxPurchaseQuantity))prints sql?

Column<b'max(Quantity) OVER (PARTITION BY CustomerId, date ORDER BY Quantity DESC NULLS LAST ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)'>

This returns a column (or expressions). We can now use this in a DataFrame select statement. Before doing so, though, we will create the purchase quantity rank. To do that, we use the dense_rank function to determine which date had the maximum purchase quantity for every customer. We use dense_rank as opposed to rank to avoid gaps in the ranking sequence when there are tied values (or in our case, duplicate rows):

In [0]:
from pyspark.sql.functions import dense_rank, rank
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)


Perform a select to view the calculated window values:

In [0]:
from pyspark.sql.functions import col

# EDIT: NOT IN TEXTBOOK - must set spark.sql.legacy.timeParserPolicy to LEGACY 
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
  .select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaseRank.alias("quantityRank"),
    purchaseDenseRank.alias("quantityDenseRank"),
    maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()





+----------+----------+--------+------------+-----------------+-------------------+
CustomerId| date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
 12346|2011-01-18| 74215| 1| 1| 74215|
 12346|2011-01-18| -74215| 2| 2| 74215|
 12347|2010-12-07| 36| 1| 1| 36|
 12347|2010-12-07| 30| 2| 2| 36|
 12347|2010-12-07| 24| 3| 3| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 12| 4| 4| 36|
 12347|2010-12-07| 6| 17| 5| 36|
 12347|2010-12-07| 6| 17| 5| 36|
+----------+----------+--------+------------+-----------------+-------------------+
only showing top 20 rows

In [0]:
dfNoNull = dfWithDate.drop()
dfNoNull.createOrReplaceTempView("dfNoNull")


Rollup - a multidimensional aggregation that performs a variety of group-by style calculations for us. 

create a rollup that looks across time (with our new Date column) and space (with the Country column) and creates a new DataFrame that includes the grand total over all dates, the grand total for each date in the DataFrame, and the subtotal for each country on each date in theDataFrame

In [0]:
rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))\
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("Date")
rolledUpDF.show()


+----------+--------------+--------------+
 Date| Country|total_quantity|
+----------+--------------+--------------+
 null| null| 5176450|
2010-12-01| Germany| 117|
2010-12-01| France| 449|
2010-12-01| Norway| 1852|
2010-12-01| EIRE| 243|
2010-12-01|United Kingdom| 23949|
2010-12-01| Netherlands| 97|
2010-12-01| Australia| 107|
2010-12-01| null| 26814|
2010-12-02|United Kingdom| 20873|
2010-12-02| null| 21023|
2010-12-02| EIRE| 4|
2010-12-02| Germany| 146|
2010-12-03| Belgium| 528|
2010-12-03| Spain| 400|
2010-12-03| Switzerland| 110|
2010-12-03| Poland| 140|
2010-12-03| Portugal| 65|
2010-12-03| France| 239|
2010-12-03| null| 14830|
+----------+--------------+--------------+
only showing top 20 rows

Where you see the null values is where you’ll find the grand totals. A null in both rollup columns specifies the grand total across both of those columns:

In [0]:
# rolledUpDF.where("Country IS NULL").show()
rolledUpDF.where("Date IS NULL && Country IS NULL").show()

+----------+-------+--------------+
 Date|Country|total_quantity|
+----------+-------+--------------+
 null| null| 5176450|
2010-12-01| null| 26814|
2010-12-02| null| 21023|
2010-12-03| null| 14830|
2010-12-05| null| 16395|
2010-12-06| null| 21419|
2010-12-07| null| 24995|
2010-12-08| null| 22741|
2010-12-09| null| 18431|
2010-12-10| null| 20297|
2010-12-12| null| 10565|
2010-12-13| null| 17623|
2010-12-14| null| 20098|
2010-12-15| null| 18229|
2010-12-16| null| 29632|
2010-12-17| null| 16069|
2010-12-19| null| 3795|
2010-12-20| null| 14965|
2010-12-21| null| 15467|
2010-12-22| null| 3192|
+----------+-------+--------------+
only showing top 20 rows

+----+-------+--------------+
Date|Country|total_quantity|
+----+-------+--------------+
null| null| 5176450|
+----+-------+--------------+

Cube - Rather than treating elements hierarchically, a cube does the same thing across all dimensions

This means that it won’t just go by date over the entire time period, but also the country. We can you make a tablethat includes the following: 
* The total across all dates and countries
* The total for each date across all countries
* The total for each country on each dateThe total for each country across all dates

In [0]:
from pyspark.sql.functions import sum

dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))\
  .select("Date", "Country", "sum(Quantity)").orderBy("Date").show()


+----+---------------+-------------+
Date| Country|sum(Quantity)|
+----+---------------+-------------+
null| Lithuania| 652|
null| Belgium| 23152|
null| Australia| 83653|
null| Denmark| 8188|
null| Spain| 26824|
null|Channel Islands| 9479|
null| Singapore| 5234|
null| Lebanon| 386|
null| Cyprus| 6317|
null| Hong Kong| 4769|
null| Finland| 10666|
null| Germany| 117448|
null| null| 5176450|
null| Norway| 19247|
null| Portugal| 16180|
null| Japan| 25218|
null| Italy| 7999|
null| Greece| 1556|
null| Iceland| 2458|
null| Netherlands| 200128|
+----+---------------+-------------+
only showing top 20 rows

**Pivots** make it possible for you to convert a row into a column. With a pivot, we can aggregate according to some function for each of those given rows and display them in an easy-to-query way:

In [0]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()
display(pivoted)

date,Australia_sum(CAST(Quantity AS BIGINT)),Australia_sum(UnitPrice),Australia_sum(CAST(CustomerID AS BIGINT)),Austria_sum(CAST(Quantity AS BIGINT)),Austria_sum(UnitPrice),Austria_sum(CAST(CustomerID AS BIGINT)),Bahrain_sum(CAST(Quantity AS BIGINT)),Bahrain_sum(UnitPrice),Bahrain_sum(CAST(CustomerID AS BIGINT)),Belgium_sum(CAST(Quantity AS BIGINT)),Belgium_sum(UnitPrice),Belgium_sum(CAST(CustomerID AS BIGINT)),Brazil_sum(CAST(Quantity AS BIGINT)),Brazil_sum(UnitPrice),Brazil_sum(CAST(CustomerID AS BIGINT)),Canada_sum(CAST(Quantity AS BIGINT)),Canada_sum(UnitPrice),Canada_sum(CAST(CustomerID AS BIGINT)),Channel Islands_sum(CAST(Quantity AS BIGINT)),Channel Islands_sum(UnitPrice),Channel Islands_sum(CAST(CustomerID AS BIGINT)),Cyprus_sum(CAST(Quantity AS BIGINT)),Cyprus_sum(UnitPrice),Cyprus_sum(CAST(CustomerID AS BIGINT)),Czech Republic_sum(CAST(Quantity AS BIGINT)),Czech Republic_sum(UnitPrice),Czech Republic_sum(CAST(CustomerID AS BIGINT)),Denmark_sum(CAST(Quantity AS BIGINT)),Denmark_sum(UnitPrice),Denmark_sum(CAST(CustomerID AS BIGINT)),EIRE_sum(CAST(Quantity AS BIGINT)),EIRE_sum(UnitPrice),EIRE_sum(CAST(CustomerID AS BIGINT)),European Community_sum(CAST(Quantity AS BIGINT)),European Community_sum(UnitPrice),European Community_sum(CAST(CustomerID AS BIGINT)),Finland_sum(CAST(Quantity AS BIGINT)),Finland_sum(UnitPrice),Finland_sum(CAST(CustomerID AS BIGINT)),France_sum(CAST(Quantity AS BIGINT)),France_sum(UnitPrice),France_sum(CAST(CustomerID AS BIGINT)),Germany_sum(CAST(Quantity AS BIGINT)),Germany_sum(UnitPrice),Germany_sum(CAST(CustomerID AS BIGINT)),Greece_sum(CAST(Quantity AS BIGINT)),Greece_sum(UnitPrice),Greece_sum(CAST(CustomerID AS BIGINT)),Hong Kong_sum(CAST(Quantity AS BIGINT)),Hong Kong_sum(UnitPrice),Hong Kong_sum(CAST(CustomerID AS BIGINT)),Iceland_sum(CAST(Quantity AS BIGINT)),Iceland_sum(UnitPrice),Iceland_sum(CAST(CustomerID AS BIGINT)),Israel_sum(CAST(Quantity AS BIGINT)),Israel_sum(UnitPrice),Israel_sum(CAST(CustomerID AS BIGINT)),Italy_sum(CAST(Quantity AS BIGINT)),Italy_sum(UnitPrice),Italy_sum(CAST(CustomerID AS BIGINT)),Japan_sum(CAST(Quantity AS BIGINT)),Japan_sum(UnitPrice),Japan_sum(CAST(CustomerID AS BIGINT)),Lebanon_sum(CAST(Quantity AS BIGINT)),Lebanon_sum(UnitPrice),Lebanon_sum(CAST(CustomerID AS BIGINT)),Lithuania_sum(CAST(Quantity AS BIGINT)),Lithuania_sum(UnitPrice),Lithuania_sum(CAST(CustomerID AS BIGINT)),Malta_sum(CAST(Quantity AS BIGINT)),Malta_sum(UnitPrice),Malta_sum(CAST(CustomerID AS BIGINT)),Netherlands_sum(CAST(Quantity AS BIGINT)),Netherlands_sum(UnitPrice),Netherlands_sum(CAST(CustomerID AS BIGINT)),Norway_sum(CAST(Quantity AS BIGINT)),Norway_sum(UnitPrice),Norway_sum(CAST(CustomerID AS BIGINT)),Poland_sum(CAST(Quantity AS BIGINT)),Poland_sum(UnitPrice),Poland_sum(CAST(CustomerID AS BIGINT)),Portugal_sum(CAST(Quantity AS BIGINT)),Portugal_sum(UnitPrice),Portugal_sum(CAST(CustomerID AS BIGINT)),RSA_sum(CAST(Quantity AS BIGINT)),RSA_sum(UnitPrice),RSA_sum(CAST(CustomerID AS BIGINT)),Saudi Arabia_sum(CAST(Quantity AS BIGINT)),Saudi Arabia_sum(UnitPrice),Saudi Arabia_sum(CAST(CustomerID AS BIGINT)),Singapore_sum(CAST(Quantity AS BIGINT)),Singapore_sum(UnitPrice),Singapore_sum(CAST(CustomerID AS BIGINT)),Spain_sum(CAST(Quantity AS BIGINT)),Spain_sum(UnitPrice),Spain_sum(CAST(CustomerID AS BIGINT)),Sweden_sum(CAST(Quantity AS BIGINT)),Sweden_sum(UnitPrice),Sweden_sum(CAST(CustomerID AS BIGINT)),Switzerland_sum(CAST(Quantity AS BIGINT)),Switzerland_sum(UnitPrice),Switzerland_sum(CAST(CustomerID AS BIGINT)),USA_sum(CAST(Quantity AS BIGINT)),USA_sum(UnitPrice),USA_sum(CAST(CustomerID AS BIGINT)),United Arab Emirates_sum(CAST(Quantity AS BIGINT)),United Arab Emirates_sum(UnitPrice),United Arab Emirates_sum(CAST(CustomerID AS BIGINT)),United Kingdom_sum(CAST(Quantity AS BIGINT)),United Kingdom_sum(UnitPrice),United Kingdom_sum(CAST(CustomerID AS BIGINT)),Unspecified_sum(CAST(Quantity AS BIGINT)),Unspecified_sum(UnitPrice),Unspecified_sum(CAST(CustomerID AS BIGINT))
2011-05-06,null,null,null